<a href="https://colab.research.google.com/github/NsiriRoua/classify_nationalities/blob/main/Classify_Nationalities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow import keras
import tensorflow as tf
import pandas as pd
import os
import re

In [ ]:
!unrar x '/content/drive/MyDrive/Colab NoQtebooks/Male-female data (1).rar'


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/drive/MyDrive/Colab Notebooks/Male-female data (1).rar


Would you like to replace the existing file Indian-Male-Names.csv
297859 bytes, modified on 2019-10-10 08:52
with a new one
297859 bytes, modified on 2019-10-10 08:52

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit Q

Program aborted


In [ ]:
male_names=pd.read_csv('Indian-Male-Names.csv')
female_names=pd.read_csv('Indian-Female-Names.csv')

In [ ]:
male_names.head()

,name,gender,race
0,barjraj,m,indian
1,ramdin verma,m,indian
2,sharat chandran,m,indian
3,birender mandal,m,indian
4,amit,m,indian


In [ ]:
female_names.head()

,name,gender,race
0,shivani,f,indian
1,isha,f,indian
2,smt shyani devi,f,indian
3,divya,f,indian
4,mansi,f,indian


# **Let's clean our data**

In [ ]:
list_to_clean=['s/o','d/o','w/o','/','&',',','-']
def clean_data(name):
	name = str(name).lower()
	name = (''.join(i for i in name if ord(i)<128)).strip()
	for repl in list_to_clean:
		name = name.replace(repl," ")
	if '@' in name:
		pos = name.find('@')
		name = name[:pos].strip()
	name = name.split(" ")
	name = " ".join([each.strip() for each in name])
	return name




In [ ]:
#Let's see an example
p='Rou@'
name =clean_data(p)
print(name)

rou


In [ ]:
def remove_records(merged_data):
	merged_data['delete'] = 0
	merged_data.loc[merged_data['name'].str.find('with') != -1,'delete'] = 1	
	merged_data.loc[merged_data['count_words']>=5,'delete']=1
	merged_data.loc[merged_data['count_words']==0,'delete']=1
	merged_data.loc[merged_data['name'].str.contains(r'\d') == True,'delete']=1
	cleaned_data = merged_data[merged_data.delete==0]
	return cleaned_data

In [ ]:
merged_data=pd.concat((female_names,male_names),axis=0)
merged_data['name'] = merged_data['name'].apply(clean_data)
merged_data['count_words'] = merged_data['name'].str.split().apply(len)

In [ ]:
cleaned_data=remove_records(merged_data)
cleaned_data.head

<bound method NDFrame.head of                       name gender    race  count_words  delete
0                  shivani      f  indian            1       0
1                     isha      f  indian            1       0
2          smt shyani devi      f  indian            3       0
3                    divya      f  indian            1       0
4                    mansi      f  indian            1       0
...                    ...    ...     ...          ...     ...
14840               buddha      m  indian            1       0
14841               mukesh      m  indian            1       0
14842                 monu      m  indian            1       0
14843  govind prasad shahu      m  indian            3       0
14844               deepak      m  indian            1       0

[29868 rows x 5 columns]>

In [ ]:
indian_cleaned_data = cleaned_data[['name','count_words']].drop_duplicates(subset='name',keep='first')
indian_cleaned_data['label'] = 'indian'
len(indian_cleaned_data)

13754

In [ ]:
pip install Faker

     |████████████████████████████████| 1.2MB 4.5MB/s 


In [ ]:
from faker import Faker
fake=Faker('en_US')
fake.name()

'Dominic Berry'

In [ ]:
profiles=[fake.name() for i in range(13754)]
non_indian_data=pd.DataFrame(profiles)
non_indian_data.head()

,0
0,Umberto Morrocco
1,Gesche Hornig MBA.
2,Bert Drubin
3,Hubert Zobel B.Eng.
4,石川 あすか


In [ ]:
len(non_indian_data)

13754

In [ ]:
non_indian_data['name'] = non_indian_data['name'].apply(clean_data)
non_indian_data['count_words'] =non_indian_data['name'].str.split().apply(len)


In [ ]:
non_indian_data=remove_records(non_indian_data)
non_indian_data = non_indian_data[['name','count_words']].drop_duplicates(subset='name',keep='first')
non_indian_data.head

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

<bound method NDFrame.head of                                  name  count_words
0                    umberto morrocco            2
1                  gesche hornig mba.            3
2                         bert drubin            2
3                 hubert zobel b.eng.            3
5                    giampaolo navone            2
...                               ...          ...
13746  univ.prof. mirjam znker b.eng.            4
13749             dott. dante alfonsi            3
13750      susanna querini modigliani            3
13751         herr stanislaw heinrich            3
13753      marie louise tsche wieloch            4

[8097 rows x 2 columns]>

# **Let's build a neural network**